# Analysis of Beams with PMEF

### Install necessary packages

In [9]:
import sys
!{sys.executable} -m pip install -q -r requirements_app.txt

In [6]:

from ipywidgets import FloatText,VBox,HBox,HTML,Layout,Image
style = {'description_width': 'initial'}
W = FloatText(
    value=0.6,
    description='Width (b):',
    disabled=False, style = style)
H = FloatText(
    value=1.0,
    description='Height (h):',
    disabled=False, style = style)
t = FloatText(
    value=0.05,
    description='Wing thickness(e):',
    disabled=False, style = style)
ta = FloatText(
    value=0.05,
    description='Core thickness (a):',
    disabled=False, style = style)

file = open("img/Perfil.png", "rb")
image = file.read()
img = Image(
    value=image,
    format='png',
    width=300,
    height=400,
)

VBox([HTML("<h1 align='center'>Profile dimensioning (m)</h2>"),

HBox([
img,
VBox([W, H, t, ta],layout=Layout(justify_content="space-around"))],layout=Layout(justify_content="center"))],
layout=Layout(justify_content="center",border='solid 4px'))

In [7]:

from ipywidgets import FloatText,VBox,HBox,HTML,Layout,Dropdown
style = {'description_width': 'initial'}
h1 = HTML(value="""<h1 align='center'>Longitudinal dimensioning of the beam (m)</h1>""")
type = Dropdown(options=["Castellated Beam","Cellular Beam",],description="Type of the beam:",disabled=False,style=style)

L = FloatText(
    value=10.0,
    description='Beam length (L):',
    disabled=False, style = style)
n = FloatText(
    value=8.0,
    description='Number of holes (n):',
    disabled=False, style = style)
d = FloatText(
    value=0.7,
    description='Hole diameter (d):',
    disabled=False, style = style)
h3 = HTML(value="<b>Size of Mesh:</b>")
Min_size = FloatText(
    value=0.3,
    description='Min size:',
    disabled=False, style = style)
Max_size = FloatText(
    value=0.4,
    description='Max size:',
    disabled=False, style = style)
file = open("img/TypeBeam.png", "rb")
image = file.read()
img = Image(
    value=image,
    format='png',
    width=250,
    height=300,
)

vbox1 = VBox([L, n, d, h3, Min_size, Max_size],layout=Layout(justify_content="space-around"))
hbox1 = HBox([img,vbox1])
hbox1.layout.justify_content="center"
vbox2 = VBox()
vbox2.children = [h1,type,hbox1]
vbox2.layout = Layout(align_items="center",border='solid 4px')
vbox2

In [8]:

from functions import *
Generate_Model(W,H,t,ta,L,n,d,type)
Generate_geo_file(t,ta,type,n,Min_size,Max_size)
%run mesh.py

In [9]:

import pyvista as pv
pv.set_jupyter_backend('pythreejs')  
mesh = pv.read("object.vtk")
p=pv.Plotter()
p.add_mesh(mesh,show_edges=True)
p.show()

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(intensity=0.25, positi…

In [10]:

from ipywidgets import FloatText,VBox,HBox,HTML,Layout
style = {'description_width': 'initial'}
h1 = HTML(value="""<h1 align='center'>Analysis with PMEF (kg,m,s)</h1>""")

Elas = FloatText(
    value=2.1e10,
    description='Modulus of elasticity:',
    disabled=False, style = style)
v = FloatText(
    value=0.2,
    description="Poisson's ratio:",
    disabled=False, style = style)
de = FloatText(
    value=7850,
    description='Density:',
    disabled=False, style = style)
F = FloatText(
    value=-1e5,
    description='Evenly distributed force:',
    disabled=False, style = style)
fix = Dropdown(options=["Start","End - Start"],description="Beam embedment type:",disabled=False,style=style)
FS = FloatText(
    value=20,
    description='Displacement scale factor:',
    disabled=False, style = style)

vbox1 =VBox([Elas,v,de])
vbox2 =VBox([F,fix,FS])
hbox1 =HBox([vbox1,vbox2])
vbox3 =VBox([h1,hbox1])
vbox3.layout = Layout(align_items="center",border='solid 2px')
vbox3


In [11]:
u = Analysis_FEM(W,H,L,Elas,v,de,F,fix, FS)

In [12]:

graph = Dropdown(options=["Ux","Uy","Uz"],description="Graphics:",disabled=False,style=style)
graph

Dropdown(description='Graphics:', options=('Ux', 'Uy', 'Uz'), style=DescriptionStyle(description_width='initia…

In [17]:

results = pv.read("result.vtk")
results["Ux"] = u[0::3]
results["Uy"] = u[1::3]
results["Uz"] = u[2::3]
pv.set_jupyter_backend('ipygany')
p=pv.Plotter()
p.add_mesh(results,scalars=results[graph.value],show_edges=True,cmap="RdYlGn")
p.show()


AppLayout(children=(VBox(children=(HTML(value='<h3></h3>'), Dropdown(description='Colormap:', options={'BrBG':…

<body>
    <div class="content">
    <!-- <input type="button" value="Actualizar" onclick="location.reload()" 
    style="font-family:Arial;font-size:8pt;width:150px;height:30px;
    background:#777777;color:#fff444;cursor:pointer;position: absolute;bottom: 0%;"/> -->
    <iframe
        width="1000"
        height="700"
        src="https://kitware.github.io/vtk-js/examples/GeometryViewer/index.html"></iframe></div>
</body>
